In [1]:
pip install -q cassio datasets langchain openai tiktoken

    extract-msg (<=0.29.*)
                 ~~~~~~~^
Note: you may need to restart the kernel to use updated packages.


In [2]:
# LangChain components to use
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings

# Support for dataset retrieval with Hugging Face
from datasets import load_dataset

# Initialize the DB connection using CassIO
import cassio
from sqlalchemy.engine import URL 

In [3]:
!pip install PyPDF2

    extract-msg (<=0.29.*)
                 ~~~~~~~^


In [4]:
from PyPDF2 import PdfReader

In [31]:
ASTRA_DB_APPLICATION_TOKEN = "AstraCS:lSCGWhBCuTaYYKTyRepoRZXZ:05c8b10ddb6e3f69d606e8afdd24788552f1bcd7083c1f0879ba97aaf4f81032" # enter the "AstraCS:..." string found in in your Token JSON file
ASTRA_DB_ID = "14882e00-9550-4fa3-853a-0daf5b13c557" # enter your Database ID

OPENAI_API_KEY = "sk-J3ZbnEqytFesD7kWKuVaT3BlbkVr9dpDbViv2R2un" # enter your OpenAI key

In [32]:
# provide the path of  pdf file/files.
pdfreader = PdfReader('/Users/jameelakthar/Downloads/c026a-summary-of-union-interim-budget-2024-25.pdf')

In [33]:
from typing_extensions import Concatenate
# read text from pdf
raw_text = ''
for i, page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        raw_text += content

In [34]:
raw_text

"WHAT IS A BUDGET?\nAs per Article 112 of the Constitution, Budget is known as Annual Financial Statement of the \nGovernment.\nThe term ‘Budget’ is not mentioned in the Constitution.\nIt is the statement of estimated receipts and expenditure of the Government in a financial year.\nUnion Budget is classified into Revenue Budget(including Tax and Non-Tax revenue receipts and \nexpenditure of the government) and Capital Budget (including capital receipts and payments of \nthe government).\nPreparation of budget:\nDepartment of Economic Affairs, Ministry of Finance is the nodal body responsible for \npreparing of the Budget.\nBudget for a year is prepared by the Budget Division of Department of Economic Affairs.\nIt is prepared broadly on the basis of detailed estimates of expenditure and receipts \nreceived from various Departments/Ministries and its own subordinate estimating \nauthorities.\nBudget is prepared on Cash Basis (i.e. whatever is expected to be actually received or paid \nun

In [35]:
cassio.init(token=ASTRA_DB_APPLICATION_TOKEN, database_id=ASTRA_DB_ID)

In [36]:
llm = OpenAI(openai_api_key=OPENAI_API_KEY)
embedding = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [ ]:
astra_vector_store = Cassandra(
    embedding=embedding,
    table_name="qa_mini_demo_1",
    session=None,
    keyspace=None,
)

In [28]:
from langchain.text_splitter import CharacterTextSplitter
# We need to split the text using Character Text Split such that it sshould not increse token size
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 800,
    chunk_overlap  = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [29]:
texts[:50]

['WHAT IS A BUDGET?\nAs per Article 112 of the Constitution, Budget is known as Annual Financial Statement of the \nGovernment.\nThe term ‘Budget’ is not mentioned in the Constitution.\nIt is the statement of estimated receipts and expenditure of the Government in a financial year.\nUnion Budget is classified into Revenue Budget(including Tax and Non-Tax revenue receipts and \nexpenditure of the government) and Capital Budget (including capital receipts and payments of \nthe government).\nPreparation of budget:\nDepartment of Economic Affairs, Ministry of Finance is the nodal body responsible for \npreparing of the Budget.\nBudget for a year is prepared by the Budget Division of Department of Economic Affairs.\nIt is prepared broadly on the basis of detailed estimates of expenditure and receipts',
 'preparing of the Budget.\nBudget for a year is prepared by the Budget Division of Department of Economic Affairs.\nIt is prepared broadly on the basis of detailed estimates of expenditure a

In [30]:
astra_vector_store.add_texts(texts[:50])

print("Inserted %i headlines." % len(texts[:50]))

astra_vector_index = VectorStoreIndexWrapper(vectorstore=astra_vector_store)

NameError: name 'astra_vector_store' is not defined

In [38]:
first_question = True
while True:
    if first_question:
        query_text = input("\nEnter your question (or type 'quit' to exit): ").strip()
    else:
        query_text = input("\nWhat's your next question (or type 'quit' to exit): ").strip()

    if query_text.lower() == "quit":
        break

    if query_text == "":
        continue

    first_question = False

    print("\nQUESTION: \"%s\"" % query_text)
    answer = astra_vector_index.query(query_text, llm=llm).strip()
    print("ANSWER: \"%s\"\n" % answer)

    print("FIRST DOCUMENTS BY RELEVANCE:")
    for doc, score in astra_vector_store.similarity_search_with_score(query_text, k=4):
        print("    [%0.4f] \"%s ...\"" % (score, doc.page_content[:84]))


Enter your question (or type 'quit' to exit): what is current GDP?

QUESTION: "what is current GDP?"


NameError: name 'astra_vector_index' is not defined